## Project Milestone 4 :   Connecting to an API and Data Wrangling  

### Milestone 4: API Data Cleaning & Transformation

**Data Source**: ACA Plan Data with County and Premium Details from [data.iowa.gov](https://data.iowa.gov)

This notebook performs data cleaning and transformation on plan-level ACA insurance premium data by region. It includes multiple numeric and categorical fields such as:

- premium_adult_individual
- metal_level
- county_name
- plan_id

The final goal is to produce a clean, human-readable dataset and evaluate the ethical implications of the transformation steps.

#### Load and Inspect the Data

In [182]:
import requests
import pandas as pd
import numpy as np

api_url = "https://data.iowa.gov/resource/h52q-pcvb.json"
response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data)
    print(f"Loaded {len(df)} records.")
else:
    raise Exception(f"API request failed with status {response.status_code}")

df.head()


Loaded 1000 records.


,plan_id,issuer_name,plan_name,plan_type,rating_area,metal_level,county_fips,county_name,premium_adult_individual,lowest_cost_rnk,...,premium_age_28,premium_family,premium_age_35,premium_age_45,premium_age_55,premium_age_62,premium_age_65,county_centroid,:@computed_region_y683_txed,:@computed_region_pva4_mz5a
0,48286IA0010016,Iowa Total Care,Everyday Gold,HMO,Rating Area 6,Gold,19019,Buchanan,330.82,3,...,359.60134,1225.35728,404.26203999999996,477.70408,737.7286,950.44586,992.46,"{'type': 'Point', 'coordinates': [-91.8378392,...",34,364
1,48286IA0010022,Iowa Total Care,Elite Silver,HMO,Rating Area 3,Silver,19133,Monona,310.55,3,...,337.56785,1150.2772,379.4921,448.4342,692.5265,892.2101500000001,931.6500000000001,"{'type': 'Point', 'coordinates': [-95.9599235,...",48,460
2,48286IA0010022,Iowa Total Care,Elite Silver,HMO,Rating Area 6,Silver,19019,Buchanan,342.98,8,...,372.81926,1270.3979199999999,419.12156,495.26312,764.8454,985.3815400000001,1028.94,"{'type': 'Point', 'coordinates': [-91.8378392,...",34,364
3,48286IA0010022,Iowa Total Care,Elite Silver,HMO,Rating Area 4,Silver,19137,Montgomery,363.06,3,...,394.64621999999997,1344.77424,443.65932,524.25864,809.6238,1043.07138,1089.18,"{'type': 'Point', 'coordinates': [-95.1563758,...",80,602
4,48286IA0010015,Iowa Total Care,Complete Gold,HMO,Rating Area 7,Gold,19109,Kossuth,344.94,3,...,374.94978,1277.65776,421.51668,498.09335999999996,769.2162,991.0126200000001,1034.82,"{'type': 'Point', 'coordinates': [-94.2067199,...",6,271


#### Step #1: Convert Premium and Ranking Fields to Numeric

The columns `premium_adult_individual`, `lowest_cost_rnk`, and `county_fips` contain numeric values but may be stored as strings. These are converted to numeric to enable filtering, computation, and statistical analysis.



In [184]:
df['premium_adult_individual'] = pd.to_numeric(df['premium_adult_individual'], errors='coerce')
df['lowest_cost_rnk'] = pd.to_numeric(df['lowest_cost_rnk'], errors='coerce')
df['county_fips'] = pd.to_numeric(df['county_fips'], errors='coerce')

#### Step #2: Drop Records with Missing Premium or County Name

Rows missing values in the `county_name` or `premium_adult_individual` fields are removed.

These fields are essential for conducting geographic and cost-based analysis, and incomplete records may introduce bias or errors in downstream process.
s.



In [186]:
print(df.columns.tolist())

['plan_id', 'issuer_name', 'plan_name', 'plan_type', 'rating_area', 'metal_level', 'county_fips', 'county_name', 'premium_adult_individual', 'lowest_cost_rnk', 'premium_age_4', 'premium_age_16', 'premium_age_28', 'premium_family', 'premium_age_35', 'premium_age_45', 'premium_age_55', 'premium_age_62', 'premium_age_65', 'county_centroid', ':@computed_region_y683_txed', ':@computed_region_pva4_mz5a']


In [187]:
# Count before dropping missing values
before_count = len(df)

# Drop rows with missing values in key columns
df_clean = df.dropna(subset=['county_name', 'premium_adult_individual'])

# Count after dropping missing values
after_count = len(df_clean)

# Print results
print(f"Records before dropping missing values: {before_count}")
print(f"Records after dropping missing values: {after_count}")
print(f"Records removed: {before_count - after_count}")

Records before dropping missing values: 1000
Records after dropping missing values: 1000
Records removed: 0


#### Step #3: Remove Outliers in Premiums

Outliers are identified as values outside 3 standard deviations from the mean of `premium_adult_individual`. These may indicate data entry errors or extreme edge cases.



In [189]:
# Calculate mean and standard deviation for outlier detection
mean = df_clean['premium_adult_individual'].mean()
std = df_clean['premium_adult_individual'].std()

# Define lower and upper bounds (3 standard deviations)
lower_bound = max(0, mean - 3 * std)
upper_bound = mean + 3 * std

# Count before removing outliers
before_count = len(df_clean)

# Filter out outliers
df_clean = df_clean[
    (df_clean['premium_adult_individual'] >= lower_bound) &
    (df_clean['premium_adult_individual'] <= upper_bound)
]

# Count after removing outliers
after_count = len(df_clean)

# Print summary
print(f"Records before removing outliers: {before_count}")
print(f"Records after removing outliers: {after_count}")
print(f"Outliers removed: {before_count - after_count}")


Records before removing outliers: 1000
Records after removing outliers: 993
Outliers removed: 7


#### Step #4: Standardize Text Fields

The fields `county_name` and `metal_level` are standardized to title case to prevent duplication due to inconsistent casing.


In [191]:
# Print BEFORE standardization
print("Before standardization:\n")
print(df_clean[['county_name', 'metal_level']].drop_duplicates().head(10))

# Apply title case
df_clean['county_name'] = df_clean['county_name'].str.title()
df_clean['metal_level'] = df_clean['metal_level'].str.title()

# Print AFTER standardization
print("\nAfter standardization:\n")
print(df_clean[['county_name', 'metal_level']].drop_duplicates().head(10))



Before standardization:

  county_name metal_level
0    Buchanan        Gold
1      Monona      Silver
2    Buchanan      Silver
3  Montgomery      Silver
4     Kossuth        Gold
5    Buchanan      Bronze
6   Winnebago        Gold
7      Shelby      Bronze
8     Guthrie        Gold
9        Linn      Silver

After standardization:

  county_name metal_level
0    Buchanan        Gold
1      Monona      Silver
2    Buchanan      Silver
3  Montgomery      Silver
4     Kossuth        Gold
5    Buchanan      Bronze
6   Winnebago        Gold
7      Shelby      Bronze
8     Guthrie        Gold
9        Linn      Silver


#### Step #5: Remove Duplicate Records

Duplicate records with the same `plan_id`, `county_name`, and `metal_level` are removed to avoid overcounting in analysis.


In [193]:
# Count before removing duplicates
before_count = len(df_clean)

# Remove duplicates based on key columns
df_clean = df_clean.drop_duplicates(subset=['plan_id', 'county_name', 'metal_level'])

# Count after removing duplicates
after_count = len(df_clean)

# Print summary
print(f"Records before removing duplicates: {before_count}")
print(f"Records after removing duplicates: {after_count}")
print(f"Duplicates removed: {before_count - after_count}")



Records before removing duplicates: 993
Records after removing duplicates: 993
Duplicates removed: 0


#### Step #6: Final Preview of Cleaned Dataset

Below is the cleaned and formatted dataset showing the first 10 records. It is free of missing data, standardized, and ready for analysis.


In [195]:
df_clean.reset_index(drop=True).head(10)

,plan_id,issuer_name,plan_name,plan_type,rating_area,metal_level,county_fips,county_name,premium_adult_individual,lowest_cost_rnk,...,premium_age_28,premium_family,premium_age_35,premium_age_45,premium_age_55,premium_age_62,premium_age_65,county_centroid,:@computed_region_y683_txed,:@computed_region_pva4_mz5a
0,48286IA0010016,Iowa Total Care,Everyday Gold,HMO,Rating Area 6,Gold,19019,Buchanan,330.82,3,...,359.60134,1225.35728,404.26203999999996,477.70408,737.7286,950.44586,992.46,"{'type': 'Point', 'coordinates': [-91.8378392,...",34,364
1,48286IA0010022,Iowa Total Care,Elite Silver,HMO,Rating Area 3,Silver,19133,Monona,310.55,3,...,337.56785,1150.2772,379.4921,448.4342,692.5265,892.2101500000001,931.6500000000001,"{'type': 'Point', 'coordinates': [-95.9599235,...",48,460
2,48286IA0010022,Iowa Total Care,Elite Silver,HMO,Rating Area 6,Silver,19019,Buchanan,342.98,8,...,372.81926,1270.3979199999999,419.12156,495.26312,764.8454,985.3815400000001,1028.94,"{'type': 'Point', 'coordinates': [-91.8378392,...",34,364
3,48286IA0010022,Iowa Total Care,Elite Silver,HMO,Rating Area 4,Silver,19137,Montgomery,363.06,3,...,394.64621999999997,1344.77424,443.65932,524.25864,809.6238,1043.07138,1089.18,"{'type': 'Point', 'coordinates': [-95.1563758,...",80,602
4,48286IA0010015,Iowa Total Care,Complete Gold,HMO,Rating Area 7,Gold,19109,Kossuth,344.94,3,...,374.94978,1277.65776,421.51668,498.09335999999996,769.2162,991.0126200000001,1034.82,"{'type': 'Point', 'coordinates': [-94.2067199,...",6,271
5,48286IA0010012,Iowa Total Care,Everyday Bronze,HMO,Rating Area 6,Bronze,19019,Buchanan,253.02,4,...,275.03274,937.18608,309.19044,365.36088,564.2346,726.9264600000001,759.0600000000001,"{'type': 'Point', 'coordinates': [-91.8378392,...",34,364
6,45819IA0010036,Oscar Insurance Company,Gold Elite | MercyOne,EPO,Rating Area 7,Gold,19189,Winnebago,385.81,4,...,419.37547,1429.04024,471.45982,557.10964,860.3563,1108.4321300000001,1157.43,"{'type': 'Point', 'coordinates': [-93.7341955,...",9,254
7,48286IA0010012,Iowa Total Care,Everyday Bronze,HMO,Rating Area 4,Bronze,19165,Shelby,267.83,2,...,291.13120999999995,992.0423199999999,327.28826,386.74652,597.2609,769.47559,803.49,"{'type': 'Point', 'coordinates': [-95.3102113,...",58,576
8,93078IA0060055,Medica Insurance Company,Medica Insure Gold Standard,EPO,Rating Area 4,Gold,19077,Guthrie,569.74,11,...,619.30738,2110.31696,696.22228,822.70456,1270.5202,1636.8630200000002,1709.22,"{'type': 'Point', 'coordinates': [-94.501055, ...",60,73
9,25896IA0370003,Wellmark Health Plan of Iowa,Wellmark Silver Traditional HMO,HMO,Rating Area 6,Silver,19113,Linn,359.22,8,...,390.47214,1330.5508800000002,438.96684000000005,518.7136800000001,801.0606,1032.03906,1077.66,"{'type': 'Point', 'coordinates': [-91.5989646,...",45,750


## Ethical Implications of Data Cleaning

#### Changes Made:
- Converted numeric columns from string to proper numeric types.
- Removed rows missing critical information (`premium_adult_individual`, `county_name`).
- Removed outliers based on a 3-sigma threshold for premiums.
- Normalized casing in categorical fields (`county_name`, `metal_level`).
- Dropped duplicate plan records.

#### Legal or Regulatory Considerations:
This dataset is publicly available through a government portal. It contains no personally identifiable information (PII) and is compliant with open data standards.

#### Risks Introduced:
- Removal of outliers may discard valid but extreme data points, especially for high-cost plans.
- Dropping missing values might bias the dataset toward more complete regions or providers.
- Casing normalization assumes text variations are non-semantic (e.g., "GOLD" = "Gold").

#### Assumptions Made:
- Outliers were defined statistically without specific healthcare domain context.
- Title casing was assumed to be safe for `metal_level` and `county_name` fields.

#### Credibility and Ethics:
Data is sourced directly from a government-hosted API. All steps performed on live or simulated representations of that data. Cleaning operations were transparent and reproducible.

#### Mitigations:
- Retain original dataset for audit purposes.
- Log transformation steps in metadata or notebook markdown.
- Where possible, consult domain experts when setting thresholds or interpreting anomalies.
